**The Data Science Method**


1.   [Problem Identification](https://medium.com/@aiden.dataminer/the-data-science-method-problem-identification-6ffcda1e5152)

2.   [Data Wrangling](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-data-collection-organization-and-definitions-d19b6ff141c4)
  * Data Collection - Collected data from wikipedia and quandl wiki price dataset. The wikipedia showed us the currect S&P 500 companies and used their ticker symbols to query quandl wiki prices.
  * Data Organization - Done using cookiecutter
  * Data Definition
  * Data Cleaning - The S&P 500 data from quandls wiki price is clean and ready for analysis use but has lost its support from Quandl community as of April 11, 2018. So we will use this dataset to setup the protfolio optimizer with proof of concept then use a different data source later for cost efficiencies.

3.   [**Exploratory Data Analysis**](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-exploratory-data-analysis-bc84d4d8d3f9)
 * Build data profile tables and plots
        - Outliers & Anomalies
 * Explore data relationships
 * Identification and creation of features

4.   [Pre-processing and Training Data Development](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967)
  * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   [Modeling](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b)
  * Create dummy or indicator features for categorical variable
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   [Documentation](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-documentation-c92c28bd45e6)

  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code
  * Finalize Documentation


First, loads the needed packages and modules into Python. Then loads the data into a pandas dataframe for ease of use.

In [14]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm



import ipywidgets as widgets
%matplotlib inline



In [15]:
# prints current directory
current_dir = os.getcwd()
print("Current Directory: ")
print(current_dir)

Current Directory: 
/Users/jb/Development/courses/springboard/ds/Assignments/Portfolio-Optimization/portopt/notebooks/exploratory


In [16]:
# prints parent directory
project_dir = os.path.abspath(os.path.join(os.path.join(current_dir, os.pardir), os.pardir))
print("Parent Directory: ")
print(project_dir)

Parent Directory: 
/Users/jb/Development/courses/springboard/ds/Assignments/Portfolio-Optimization/portopt


In [17]:
print(os.listdir())

['3.0-jujbates-S&P500-PO_eda.ipynb', '.DS_Store', '.ipynb_checkpoints', '2.0-jujbates-S&P500-PO_data_wrangling.ipynb']


In [18]:
plt.style.use('dark_background')
c = ['white', 'springgreen', 'fuchsia', 'lightcoral', 'red'] # Color
s = [24, 20, 16, 12]  # Size
w = [0.75, 1, 1.25, 1.50] # Line Width
ga = 0.10 # Grid Alpha

In [40]:

wiki_df = pd.read_csv(project_dir + '/data/interim/'+ 'wiki_sp500_interim.csv', index_col=['Symbol'])
slick_df = pd.read_csv(project_dir + '/data/interim/'+ 'slick_sp500_interim.csv', index_col=['#'])
sp500_adj_close_df = pd.read_csv(project_dir + '/data/interim/'+ 'yahoo_sp500_adj_close_interim.csv', index_col=['date'])
sp500_df = pd.read_csv(project_dir + '/data/interim/'+ 'yahoo_sp500_shares_outstanding_interim.csv', index_col=['symbol'])
sp500_index_adj_close_df = pd.read_csv(project_dir + '/data/interim/'+ 'yahoo_sp500_index_adj_close_interim.csv', index_col=['date'])



In [41]:
wiki_df.head()

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,,
MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [42]:
wiki_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 505 entries, MMM to ZTS
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Security               505 non-null    object
 1   SEC filings            505 non-null    object
 2   GICS Sector            505 non-null    object
 3   GICS Sub-Industry      505 non-null    object
 4   Headquarters Location  505 non-null    object
 5   Date first added       450 non-null    object
 6   CIK                    505 non-null    int64 
 7   Founded                496 non-null    object
dtypes: int64(1), object(7)
memory usage: 35.5+ KB


In [43]:
slick_df.head()


,Company,Symbol,Weight,Price,Chg,% Chg
#,,,,,,
1,Apple Inc.,AAPL,6.499271,114.48,-1.27,(-1.10%)
2,Microsoft Corporation,MSFT,5.679779,215.67,0.78,(0.36%)
3,Amazon.com Inc.,AMZN,4.723456,3178.26,1.86,(0.06%)
4,Facebook Inc. Class A,FB,2.335458,282.94,4.82,(1.73%)
5,Alphabet Inc. Class A,GOOGL,1.686059,1622.11,15.45,(0.96%)


In [44]:
slick_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505 entries, 1 to 505
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Company  505 non-null    object 
 1   Symbol   505 non-null    object 
 2   Weight   505 non-null    float64
 3   Price    505 non-null    float64
 4   Chg      505 non-null    float64
 5   % Chg    505 non-null    object 
dtypes: float64(3), object(3)
memory usage: 27.6+ KB


In [45]:
sp500_adj_close_df.head()


,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,179.635086,67.305016,78.864014,309.959991,136.644974,46.350380,224.570007,18.830000,156.437698,13.369433,...,100.250526,46.041862,18.711531,84.600266,65.003914,88.636864,156.240005,100.891823,39.315971,83.466866
2019-01-03,172.871124,64.128601,76.265549,302.290009,131.979675,44.704514,215.699997,17.049999,161.321426,13.360003,...,99.280663,45.860447,18.302540,81.414970,62.775536,86.407372,146.880005,99.066940,39.098495,80.581871
2019-01-04,179.983185,65.958916,78.722588,313.440002,137.111526,46.488358,226.190002,19.000000,157.319321,13.803139,...,103.284927,46.309223,19.008987,85.184715,65.424149,88.656235,152.970001,102.450378,40.299343,83.743507
2019-01-07,179.569214,66.946693,79.871574,314.799988,137.587784,47.799141,229.259995,20.570000,159.627472,13.869138,...,105.119019,46.108707,19.417980,87.434891,64.456589,88.559319,155.289993,102.489838,40.460091,84.247383
2019-01-08,180.321808,66.142906,80.242790,318.420013,141.067322,49.247898,232.679993,20.750000,158.518005,14.189702,...,108.412689,46.643410,20.059361,85.769180,65.062546,88.384834,156.330002,100.191460,40.828854,85.502151


In [46]:
sp500_adj_close_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 252 entries, 2019-01-02 to 2019-12-31
Columns: 496 entries, MMM to ZTS
dtypes: float64(496)
memory usage: 978.5+ KB


In [47]:
sp500_index_adj_close_df.head()


,^GSPC
date,
2019-01-02,2510.030029
2019-01-03,2447.889893
2019-01-04,2531.939941
2019-01-07,2549.689941
2019-01-08,2574.409912


In [48]:
sp500_adj_close_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 252 entries, 2019-01-02 to 2019-12-31
Columns: 496 entries, MMM to ZTS
dtypes: float64(496)
memory usage: 978.5+ KB


In [49]:
sp500_df.head()


,sharesOutstanding
symbol,
ATVI,771872000
ABBV,1764829952
ACN,635000000
ABMD,45047300
MMM,576019008


In [50]:
sp500_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 504 entries, ATVI to XYL
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   sharesOutstanding  504 non-null    int64
dtypes: int64(1)
memory usage: 7.9+ KB


In [51]:
sp500_adj_close_df.index = pd.to_datetime(sp500_adj_close_df.index)
sp500_adj_close_df.index

DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07',
               '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11',
               '2019-01-14', '2019-01-15',
               ...
               '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20',
               '2019-12-23', '2019-12-24', '2019-12-26', '2019-12-27',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', name='date', length=252, freq=None)

In [53]:
sp500_index_adj_close_df.index = pd.to_datetime(sp500_index_adj_close_df.index)
sp500_index_adj_close_df.index

DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07',
               '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11',
               '2019-01-14', '2019-01-15',
               ...
               '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20',
               '2019-12-23', '2019-12-24', '2019-12-26', '2019-12-27',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', name='date', length=252, freq=None)

NameError: name 'interact' is not defined

In [ ]:

def plot_adj_close_daily_return_hist(ticker):
    
    sp_daily_lag_df = sp_df.shift(1)
    sp_dr_df = (sp_daily_lag_df/sp_df) - 1

    
    mean = sp_dr_df.mean()
    
    
    std = sp_dr_df.std()

    fig, (ax1, ax2) = plt.subplots(2, figsize=(20, 16))
    ax1.set_title(f"S&P 500 Stocks ({ticker})", fontsize=s[1])
    ax1.set_ylabel(f"Price", fontsize=s[2])
    ax1.plot(sp_df.index.values, sp_df[ticker], label=ticker, linewidth=w[3], color=c[2])
    
    ax1.legend(loc='upper left', title = 'Tickers')
    ax1.grid(alpha=ga)
    
    ax2.set_title(f"S&P 500 Stocks ({ticker})", fontsize=s[1])
    ax2.set_ylabel(f"Price", fontsize=s[2])
    
    ax2.hist(sp_dr_df[ticker], bins=50, label=ticker, linewidth=w[3], color=c[1])
    
    ax2.axvline(mean[ticker], color='red', linestyle='dashed', linewidth=2)
    #to plot the std line we plot both the positive and negative values 
    ax2.axvline(std[ticker], color='g', linestyle='dashed', linewidth=2)
    ax2.axvline(-std[ticker], color='g', linestyle='dashed', linewidth=2)

    ax2.legend(loc='upper left', title = 'Tickers')
    ax2.grid(alpha=ga)
    
    plt.xticks(fontsize=s[3])
    plt.yticks(fontsize=s[3])
    plt.show()
    
    return 

interact(plot_adj_close_daily_return_hist, 
         ticker=sp_df);



#### Cumulative Returns

In [ ]:
# Cumulative Return
def cumulative_return(df):
    return (df.iloc[-1,:] / df.iloc[0,:]) - 1

sp_cr = cumulative_return(sp_df)
sp_cr.head()

In [ ]:
# Calculate percentage returns
returns = sp_df.pct_change()

In [ ]:
# Calculate individual mean returns 
meanDailyReturns = returns.mean()

In [ ]:
# Define weights for the portfolio
weights = np.array([100 / sp_df.shape[1] for x in range(sp_df.shape[1])])

In [ ]:
# Calculate expected portfolio performance
portReturn = np.sum(meanDailyReturns * weights)

In [ ]:
# Create portfolio returns column
returns['Portfolio'] = returns.dot(weights)

In [ ]:
# Calculate cumulative returns
daily_cum_ret=(1+returns).cumprod()

In [ ]:
# Plot the portfolio cumulative returns only
# fig, ax = plt.subplots()
# ax.plot(daily_cum_ret.index, daily_cum_ret.Portfolio, color=c[2], label="Portfolio")
# plt.legend()
# plt.show()



fig, ax = plt.subplots(1, figsize=(20, 8))
ax.set_title("title", fontsize=s[1])
ax.set_ylabel("ylabel", fontsize=s[2])
ax.plot(daily_cum_ret.index, daily_cum_ret.Portfolio, color=c[2], label="Portfolio")

ax.legend(loc='upper right')
ax.grid(alpha=ga)

In [ ]:
# Get percentage daily returns
daily_returns = sp_df.pct_change()

# Calculate the covariance matrix 
cov_matrix = (daily_returns.cov())*252

# Calculate the portfolio variance
portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))

# Print the result
print(str(np.round(portfolio_variance, 4) * 100) + '%')

# Calculate the standard deviation by taking the square root
portfolio_standard_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# Print the results 
print(str(np.round(portfolio_standard_dev, 4) * 100) + '%')


In [ ]:
# Calculate total rate of return from start to end
total_return = (sp_df.iloc[-1,:] - sp_df.iloc[0,:]) / sp_df.iloc[0,:]
print(total_return)

In [ ]:
# Annualize return
annualized_return = ((1 + total_return)**(12 / 12)) - 1
annualized_return['Portfolio'] = annualized_return.dot(weights)
print (annualized_return)

In [ ]:
# risk-free rate
rfr = 0

# Calculate annualized volatility from the standard deviation
vol_sp500 = returns['Portfolio'].std() * np.sqrt(252)

# Calculate the Sharpe ratio 
sharpe_ratio = ((annualized_return['Portfolio'] - rfr) / vol_sp500)
print (sharpe_ratio)

In [ ]:

fig, (ax1, ax2) = plt.subplots(2, figsize=(20, 16))
ax1.set_title(f"S&P 500 Stocks Returns (Portfolio)", fontsize=s[1])
ax1.set_ylabel(f"Percentage of Change", fontsize=s[2])
ax1.plot(returns.index.values, returns.Portfolio*100, label='Portfolio', linewidth=w[3], color=c[2])

ax1.legend(loc='upper left')
ax1.grid(alpha=ga)


returns_mean = returns.mean()
returns_std = returns.std()

ax2.set_title(f"S&P 500 Stocks Returns Distribution (Portfolio)", fontsize=s[1])
ax2.set_ylabel(f"Frequency", fontsize=s[2])
ax2.set_xlabel(f"Percentage of Change", fontsize=s[2])

ax2.hist(returns.Portfolio*100, bins=50, label='Portfolio', linewidth=w[3], color=c[1])

ax2.axvline(returns_mean.Portfolio*100, color='red', linestyle='dashed', linewidth=2)
#to plot the std line we plot both the positive and negative values 
ax2.axvline(returns_std.Portfolio*100, color='g', linestyle='dashed', linewidth=2)
ax2.axvline(-returns_std.Portfolio*100, color='g', linestyle='dashed', linewidth=2)

ax2.legend(loc='upper left', title = 'Tickers')
ax2.grid(alpha=ga)

plt.xticks(fontsize=s[3])
plt.yticks(fontsize=s[3])
plt.show()


In [ ]:
returns_skew = returns.skew()
returns_kurtosis = returns.kurtosis()

print('Returns Skew: ')
print(returns_skew.Portfolio)
print('---------------------------------')
print('Returns Kurtosis: ')
print(returns_kurtosis.Portfolio)

In [ ]:
target = 0

# Create a downside return column with the negative returns only
downside_returns = returns.loc[returns.Portfolio < target]

# Calculate expected return and std dev of downside
expected_return_mean = returns.Portfolio.mean()
downside_returns_std = downside_returns.Portfolio.std()

# Calculate the sortino ratio
sortino_ratio = (expected_return_mean - rfr)/downside_returns_std

# Print the results
print("Expected return  : ", expected_return_mean*100)
print("Downside risk   : ", downside_returns_std*100)
print("Sortino ratio : ", sortino_ratio)

In [ ]:
# Calculate the max value 
roll_max = sp_df.rolling(center=False, min_periods=1, window=252).max()

# Calculate the daily draw-down relative to the max
daily_draw_down = sp_df/roll_max - 1.0

# Calculate the minimum (negative) daily draw-down
max_daily_draw_down = daily_draw_down.rolling(center=False, min_periods=1, window=252).min()





In [ ]:
def plot_daily_down(ticker):

    fig, ax1 = plt.subplots(1, figsize=(20, 16))
    ax1.set_title(f"S&P 500 Stocks ({ticker})", fontsize=s[1])
    ax1.set_ylabel(f"Price", fontsize=s[2])
    # Plot the results
    ax1.plot(sp_df.index.values, daily_draw_down[ticker], label='Daily drawdown')
    ax1.plot(sp_df.index.values, max_daily_draw_down[ticker], label='Maximum daily drawdown in time-window')

    ax1.legend(loc='upper right', title = 'Tickers')
    ax1.grid(alpha=ga)
    
    return 

interact(plot_daily_down, 
         ticker=sp_df);



In [ ]:
weights.sum()

In [ ]:
# Calculate return of the portfolio
# total_return_pf = (weights*portfolio_data['mean_return']).sum()